In [8]:
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'..')))
from utils import load_tensor, save_model
from models import BiLSTM, CNN_BiLSTM
from evaluation_functions import class_accuracy, class_f1_score

In [9]:
def remap_targets(target_tensor: torch.TensorType, old_le, new_le):
    inverse_tensor = old_le.inverse_transform(target_tensor.long())
    for idx, label in enumerate(inverse_tensor):
        if label == 'ARG_RESPONDENT' or label == 'ARG_PETITIONER':
            inverse_tensor[idx] = "ARG"
        elif label == 'PRE_NOT_RELIED' or label == 'PRE_RELIED':
            inverse_tensor[idx] = 'PRE'
    new_tensor = torch.tensor(new_le.transform(inverse_tensor))
    return new_tensor

In [10]:
list_of_targets_old = ['ISSUE', 'FAC', 'NONE', 'ARG_PETITIONER', 'PRE_NOT_RELIED', 'STA', 'RPC', 'ARG_RESPONDENT', 'PREAMBLE', 'ANALYSIS', 'RLC', 'PRE_RELIED', 'RATIO']
label_encoder_old = LabelEncoder().fit(list_of_targets_old)
list_of_targets_new = ['ISSUE', 'FAC', 'NONE', 'ARG', 'PRE', 'STA', 'RPC', 'PREAMBLE', 'ANALYSIS', 'RLC', 'RATIO']
label_encoder_new = LabelEncoder().fit(list_of_targets_new)

In [11]:
sample_input, sample_target = None, None
for idx in range(246):
    if sample_input is None:
        sample_input = load_tensor(filepath=f"../train_document/doc_{idx}/embedding")
        sample_target = load_tensor(filepath=f"../train_document/doc_{idx}/label")
    else:
        sample_input = torch.cat((sample_input,load_tensor(filepath=f"../train_document/doc_{idx}/embedding")), dim=0)
        sample_target = torch.cat((sample_target,load_tensor(filepath=f"../train_document/doc_{idx}/label")), dim=0)

In [12]:
sample_target.size()

torch.Size([28864])

In [13]:
remapped_target = remap_targets(sample_target, label_encoder_old, label_encoder_new)

In [14]:
from sklearn.utils.class_weight import compute_class_weight
    
class_weights = compute_class_weight(class_weight = "balanced",
                                    classes = np.unique(remapped_target.numpy()),
                                    y = remapped_target.numpy())
class_weights = torch.FloatTensor(class_weights)
class_weights

tensor([0.2460, 1.3107, 0.4584, 7.1499, 1.8544, 1.6639, 0.6347, 3.8990, 3.5033,
        2.4341, 5.4895])

In [17]:
def calculate_confusion_matrix(test_emb, test_labels, model, num_labels = 11):
    """

    Parameters:

    Returns:
    """
    model.eval()
    output = model(test_emb)
    return confusion_matrix(output, test_labels, num_labels)

def confusion_matrix(y_pred, y_true, num_classes):
    """
    Create a confusion matrix for label encodings in PyTorch.

    Parameters:
    y_pred (torch.Tensor): Predicted labels tensor.
    y_true (torch.Tensor): True labels tensor.
    num_classes (int): Number of classes.

    Returns:
    numpy.ndarray: Confusion matrix.
    """ 
    conf_matrix = np.zeros((num_classes, num_classes), dtype=np.int64)

    y_pred_np = y_pred.argmax(dim=1).cpu().numpy()
    y_true_np = y_true.cpu().numpy()

    for pred, true in zip(y_pred_np, y_true_np):
        conf_matrix[pred, true] += 1

    return conf_matrix

# BERT-Base

## BiLSTM

In [19]:
model1 = BiLSTM(hidden_size=128, dropout= 0.25, output_size= 11)
optimizer = torch.optim.Adam(model1.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model1.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model1(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.03:
        break
# batch_loss.append(loss.item())


-----------------------------------------Starting Training------------------------------------------


  1%|          | 2/246 [00:00<00:14, 16.83it/s]

100%|██████████| 246/246 [00:11<00:00, 22.25it/s]


Epoch: 1 	 Loss: 1.41150


100%|██████████| 246/246 [00:08<00:00, 27.47it/s]


...

Epoch: 63 	 Loss: 0.05300


100%|██████████| 246/246 [00:07<00:00, 32.00it/s]


Epoch: 64 	 Loss: 0.04585


100%|██████████| 246/246 [00:07<00:00, 33.19it/s]

Epoch: 65 	 Loss: 0.02967


In [20]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model1, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))
# result.append((config, (average_accuracy, average_f1)))

Accuracies: [0.81427072 0.41414141 0.84507042 0.80392157 0.93157895 0.49206349
 0.99603175 0.57377049 0.67857143 0.84946237 0.53125   ] 
 Average acccuracy: 0.7209211455239293
F1 Scores: [0.8049792  0.40394084 0.83696595 0.81188114 0.9315789  0.61386134
 0.9960317  0.52631574 0.44186042 0.87292813 0.557377  ] 
 Average F1: 0.7088836671511293


## CNN-BiLSTM

In [21]:
model2 = CNN_BiLSTM(hidden_size=128, dropout= 0.25, output_size= 11)
optimizer = torch.optim.Adam(model2.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model2.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model2(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.03:
        break
# batch_loss.append(loss.item())

-----------------------------------------Starting Training------------------------------------------


  1%|          | 2/246 [00:00<00:13, 18.56it/s]

100%|██████████| 246/246 [00:08<00:00, 27.69it/s]


Epoch: 1 	 Loss: 1.57144


100%|██████████| 246/246 [00:08<00:00, 30.27it/s]


...

Epoch: 74 	 Loss: 0.04420


100%|██████████| 246/246 [00:08<00:00, 27.67it/s]


Epoch: 75 	 Loss: 0.03215


100%|██████████| 246/246 [00:08<00:00, 29.58it/s]

Epoch: 76 	 Loss: 0.02146


In [22]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model2, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

Accuracies: [0.79356846 0.37878788 0.78288431 0.73076923 0.93333333 0.65405405
 0.98242188 0.59375    0.62686567 0.85714286 0.5       ] 
 Average acccuracy: 0.7121434250979548
F1 Scores: [0.78906648 0.2941176  0.81652888 0.74509799 0.90810806 0.71810084
 0.99015743 0.55882348 0.45901635 0.90322576 0.57971009] 
 Average F1: 0.7056320859173362


# LEGAL-BERT

## BiLSTM

In [23]:
model3 = BiLSTM(hidden_size=128, dropout= 0.25, output_size= 11)
optimizer = torch.optim.Adam(model3.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model3.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}_legal/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}_legal/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model3(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.1:
        break

-----------------------------------------Starting Training------------------------------------------


  1%|          | 2/246 [00:00<00:17, 14.13it/s]

100%|██████████| 246/246 [00:08<00:00, 29.79it/s]


Epoch: 1 	 Loss: 2.22225


100%|██████████| 246/246 [00:07<00:00, 34.01it/s]


...

Epoch: 98 	 Loss: 0.20081


100%|██████████| 246/246 [00:07<00:00, 34.20it/s]


Epoch: 99 	 Loss: 0.18144


100%|██████████| 246/246 [00:07<00:00, 35.06it/s]

Epoch: 100 	 Loss: 0.19187


In [24]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}_legal/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}_legal/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model3, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

Accuracies: [0.66444233 0.0952381  0.52470588 0.7        0.76315789 0.2020202
 0.83030303 0.3125     0.33333333 0.49193548 0.3       ] 
 Average acccuracy: 0.47433056798910883
F1 Scores: [0.68873513 0.07185624 0.62421269 0.2333333  0.43609018 0.1593625
 0.82282277 0.32894732 0.01680672 0.57547165 0.15384611] 
 Average F1: 0.3737713290617482


## CNN-BiLSTM

In [25]:
model4 = CNN_BiLSTM(hidden_size=128, dropout= 0.25, output_size= 11)
optimizer = torch.optim.Adam(model4.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model4.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}_legal/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}_legal/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model4(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.1:
        break

-----------------------------------------Starting Training------------------------------------------


  1%|          | 2/246 [00:00<00:19, 12.63it/s]

100%|██████████| 246/246 [00:08<00:00, 27.54it/s]


Epoch: 1 	 Loss: 2.24902


100%|██████████| 246/246 [00:08<00:00, 29.44it/s]


...

Epoch: 98 	 Loss: 0.12097


100%|██████████| 246/246 [00:08<00:00, 28.07it/s]


Epoch: 99 	 Loss: 0.11448


100%|██████████| 246/246 [00:08<00:00, 28.18it/s]

Epoch: 100 	 Loss: 0.11275


In [26]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}_legal/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}_legal/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model4, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

Accuracies: [0.65126812 0.0212766  0.55426918 0.34090909 0.54022989 0.11403509
 0.82916667 0.31325301 0.05660377 0.52941176 0.11111111] 
 Average acccuracy: 0.3692303889634425
F1 Scores: [0.69167864 0.01324499 0.60314956 0.31914889 0.33935014 0.09774431
 0.80894304 0.33548382 0.03550292 0.568421   0.14457827] 
 Average F1: 0.3597495971500088


In [27]:
torch.save(model1, 'bert-base-bilstm.pth')
torch.save(model2, 'bert-base-cnnbilstm.pth')
torch.save(model3, 'bert-legal-bilstm.pth')
torch.save(model4, 'bert-legal-cnnbilstm.pth')

# InLEGAL BERT

In [28]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'..')))

from Dataset_Reader import Dataset_Reader
from utils import read_json, data_to_embeddings, save_tensor, label_encode, organize_data
from utils import document_max_length, write_dictionary_to_json

from main import TRAIN_DATA_PATH, TEST_DATA_PATH

In [29]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")

tokenizer_config.json: 100%|██████████| 516/516 [00:00<00:00, 419kB/s]
vocab.txt: 100%|██████████| 222k/222k [00:00<00:00, 1.06MB/s]
config.json: 100%|██████████| 671/671 [00:00<00:00, 492kB/s]
pytorch_model.bin: 100%|██████████| 534M/534M [00:51<00:00, 10.5MB/s] 
/home/kshitij/.local/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [32]:
train_data = Dataset_Reader('../data/train.json')
test_data = Dataset_Reader('../data/dev.json')

print(f"Number of sentences in training data: {len(train_data.texts)}")
print(f"Number of sentences in test data: {len(test_data.texts)}")

# Manually defining labels for convenience
list_of_targets = ['ISSUE', 'FAC', 'NONE', 'ARG_PETITIONER', 'PRE_NOT_RELIED',
                'STA', 'RPC', 'ARG_RESPONDENT', 'PREAMBLE', 'ANALYSIS', 'RLC', 'PRE_RELIED', 'RATIO']

# Numerically encode labels
label_encoder = label_encode(list_of_targets)


#Compute the maximum sentence length for each document in the training and test data (to ensure all embeddings will be the same size within a document)
max_length_dict_TRAIN = document_max_length(train_data, tokenizer=tokenizer)
max_length_dict_TEST = document_max_length(test_data, tokenizer=tokenizer)

# # To same time during training process, write these documents to json file
# write_dictionary_to_json(max_length_dict_TRAIN, 'max_length_dicts/max_length_train.json')
# write_dictionary_to_json(max_length_dict_TEST, 'max_length_dicts/max_length_test.json')


#retrieve max_length dictionaries to compute word embeddings
# max_length_dict_TRAIN = read_json('max_length_dicts/max_length_train.json', reading_max_length=True)
# max_length_dict_TEST = read_json('max_length_dicts/max_length_test.json', reading_max_length=True)


#organize and process data

train_doc_idxs, train_batched_texts, train_batched_labels = organize_data(train_data, batch_size= 1) 
test_doc_idxs, test_batched_texts, test_batched_labels = organize_data(test_data, batch_size= 1) 

for idx, train_idx in tqdm(enumerate(train_doc_idxs)):
    TRAIN_emb, TRAIN_labels = data_to_embeddings(train_idx, train_batched_texts[idx], train_batched_labels[idx],
                                                label_encoder,max_length_dict_TRAIN, tokenizer=tokenizer,
                                                emb_model=model)
    save_tensor(TRAIN_emb, 'train_document/doc_'+str(idx)+'_inlegal/',"embedding")
    save_tensor(TRAIN_labels, 'train_document/doc_'+str(idx)+'_inlegal/',"label")
    

for idx, test_idx in tqdm(enumerate(test_doc_idxs)):
    TEST_emb, TEST_labels = data_to_embeddings(test_idx, test_batched_texts[idx], test_batched_labels[idx],
                                            label_encoder,max_length_dict_TEST, tokenizer=tokenizer,
                                            emb_model=model)
    save_tensor(TEST_emb, 'test_document/doc_'+str(idx)+'_inlegal/',"embedding")
    save_tensor(TEST_labels, 'test_document/doc_'+str(idx)+'_inlegal/',"label")
    

Number of sentences in training data: 28986
Number of sentences in test data: 2890


1it [00:32, 32.08s/it]

X_train size: torch.Size([91, 1, 768])	Y_train size: torch.Size([91])
Tensor saved to 'train_document/doc_0_inlegal/embedding'
Tensor saved to 'train_document/doc_0_inlegal/label'


2it [01:07, 33.86s/it]

X_train size: torch.Size([72, 1, 768])	Y_train size: torch.Size([72])
Tensor saved to 'train_document/doc_1_inlegal/embedding'
Tensor saved to 'train_document/doc_1_inlegal/label'


...

X_train size: torch.Size([54, 1, 768])	Y_train size: torch.Size([54])
Tensor saved to 'test_document/doc_26_inlegal/embedding'
Tensor saved to 'test_document/doc_26_inlegal/label'


28it [20:09, 30.42s/it]

X_train size: torch.Size([43, 1, 768])	Y_train size: torch.Size([43])
Tensor saved to 'test_document/doc_27_inlegal/embedding'
Tensor saved to 'test_document/doc_27_inlegal/label'


29it [20:26, 42.30s/it]

X_train size: torch.Size([59, 1, 768])	Y_train size: torch.Size([59])
Tensor saved to 'test_document/doc_28_inlegal/embedding'
Tensor saved to 'test_document/doc_28_inlegal/label'


## BiLSTM

In [51]:
model5 = BiLSTM(hidden_size=128, dropout= 0.3, output_size= 11)
optimizer = torch.optim.Adam(model5.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model5.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"train_document/doc_{idx}_inlegal/embedding")
        TRAIN_labels = load_tensor(filepath=f"train_document/doc_{idx}_inlegal/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model5(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.05:
        break

-----------------------------------------Starting Training------------------------------------------


  1%|          | 2/246 [00:00<00:19, 12.35it/s]

100%|██████████| 246/246 [00:11<00:00, 20.92it/s]


Epoch: 1 	 Loss: 1.42305


100%|██████████| 246/246 [00:10<00:00, 23.23it/s]


...

100%|██████████| 246/246 [00:11<00:00, 22.28it/s]


Epoch: 4 	 Loss: 0.55399


100%|██████████| 246/246 [00:09<00:00, 24.94it/s]


Epoch: 5 	 Loss: 0.46063


 70%|██████▉   | 171/246 [00:06<00:02, 34.84it/s]

In [ ]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"test_document/doc_{i}_inlegal/embedding")
    TEST_labels = load_tensor(filepath=f"test_document/doc_{i}_inlegal/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model5, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

Accuracies: [0.78586724 0.64       0.73699422 0.69090909 0.86       0.53896104
 0.9960396  0.47826087 0.59574468 0.88888889 0.38297872] 
 Average acccuracy: 0.6904222139891361
F1 Scores: [0.76898895 0.5363128  0.80251765 0.72380947 0.88205123 0.54248361
 0.99702671 0.46808506 0.34355824 0.85207095 0.47368416] 
 Average F1: 0.671871712604626


## CNN-BiLSTM

In [ ]:
model6 = BiLSTM(hidden_size=128, dropout= 0.3, output_size= 11)
optimizer = torch.optim.Adam(model6.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model6.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"train_document/doc_{idx}_inlegal/embedding")
        TRAIN_labels = load_tensor(filepath=f"train_document/doc_{idx}_inlegal/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model6(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.05:
        break

In [ ]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"test_document/doc_{i}_inlegal/embedding")
    TEST_labels = load_tensor(filepath=f"test_document/doc_{i}_inlegal/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model6, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))